In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
# 1) CSV laden und splitten
#    (90% Training, 10% Validation)
ds = load_dataset("csv", data_files="sentences_cleaned.csv", split="train")
ds = ds.train_test_split(test_size=0.2, seed=42)
train_ds, eval_ds = ds["train"], ds["test"]

In [3]:
# 2) Labels mappen und Originalspalte droppen
label_list = ["unfreundlich","neutral","freundlich"]
label2id   = {lab:i for i,lab in enumerate(label_list)}
id2label   = {i:lab for lab,i in label2id.items()}

def map_labels(ex):
    return {"labels": label2id[ex["label"]]}

train_ds = train_ds.map(map_labels, remove_columns=["label"])
eval_ds  = eval_ds.map(map_labels, remove_columns=["label"])

In [4]:
# 3) Tokenizer laden und Texte tokenizen (und "sentence" entfernen)
MODEL_NAME = "dvm1983/TinyBERT_General_4L_312D_de"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def tokenize_fn(ex):
    return tokenizer(
        ex["sentence"], padding="max_length", truncation=True, max_length=128
    )

train_ds = train_ds.map(tokenize_fn, batched=True, remove_columns=["sentence"])
eval_ds  = eval_ds.map(tokenize_fn, batched=True, remove_columns=["sentence"])

In [5]:
# 4) Data collator für dynamisches Padding
#    stellt sicher, dass alle Batches rechteckige Tensors sind
data_collator = DataCollatorWithPadding(tokenizer)

In [6]:
# 5) Format für PyTorch-Tensors
train_ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])
eval_ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

In [7]:
# 6) Modell laden (Classification Head wird neu initialisiert)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_list),
    label2id=label2id,
    id2label=id2label,
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dvm1983/TinyBERT_General_4L_312D_de and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# 7) TrainingArguments konfigurieren
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.05,
    warmup_ratio=0.2,
    evaluation_strategy="steps",
    eval_steps=250,
    save_strategy="steps",
    save_steps=250,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to=["none"],        # keine externen Logger
    dataloader_num_workers=0,    # kein Multiprocessing in DataLoader
    no_cuda=True,                # CPU-only (MPS/GPU umgehen)
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/training_args.py:1560: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [9]:
# 8) Metrics-Funktion: entpacke evtl. Tuple in p.predictions
def compute_metrics(p):
    logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds  = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

In [10]:
# 9) Trainer initialisieren
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [11]:
# 10) Optional: Ein Batch-Shape-Debug ausgeben
batch = next(iter(trainer.get_train_dataloader()))
print("Batch-Shapes:", {k: v.shape for k,v in batch.items()})

Batch-Shapes: {'labels': torch.Size([16]), 'input_ids': torch.Size([16, 128]), 'attention_mask': torch.Size([16, 128])}


In [12]:
# 11) Training starten
trainer.train()

  0%|          | 0/1758 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

{'eval_loss': 0.4258507192134857, 'eval_accuracy': 0.9397693293464332, 'eval_runtime': 105.0811, 'eval_samples_per_second': 22.278, 'eval_steps_per_second': 0.704, 'epoch': 0.43}
{'loss': 0.5812, 'grad_norm': 5.0923991203308105, 'learning_rate': 1.7894736842105264e-05, 'epoch': 0.85}


  0%|          | 0/74 [00:00<?, ?it/s]

{'eval_loss': 0.11504026502370834, 'eval_accuracy': 0.9773601025202905, 'eval_runtime': 102.0835, 'eval_samples_per_second': 22.932, 'eval_steps_per_second': 0.725, 'epoch': 0.85}


...

  0%|          | 0/74 [00:00<?, ?it/s]

{'eval_loss': 0.04453492537140846, 'eval_accuracy': 0.9897479709525844, 'eval_runtime': 102.5108, 'eval_samples_per_second': 22.837, 'eval_steps_per_second': 0.722, 'epoch': 2.56}


  0%|          | 0/74 [00:00<?, ?it/s]

{'eval_loss': 0.03899088874459267, 'eval_accuracy': 0.9918838103374626, 'eval_runtime': 98.2029, 'eval_samples_per_second': 23.838, 'eval_steps_per_second': 0.754, 'epoch': 2.99}
{'train_runtime': 1272.5217, 'train_samples_per_second': 22.069, 'train_steps_per_second': 1.382, 'train_loss': 0.20711825677830475, 'epoch': 3.0}


TrainOutput(global_step=1758, training_loss=0.20711825677830475, metrics={'train_runtime': 1272.5217, 'train_samples_per_second': 22.069, 'train_steps_per_second': 1.382, 'total_flos': 100677258443520.0, 'train_loss': 0.20711825677830475, 'epoch': 3.0})

In [13]:
trainer.save_model("tinybert-german-finetuned")
print("Fertig! Modell liegt in ./tinybert-german-finetuned")

Fertig! Modell liegt in ./tinybert-german-finetuned


In [17]:
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding
)
import torch
from torch.utils.data import DataLoader
from scipy.special import softmax
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# 1) Test-Set definieren
# 1) Test-Set definieren (erweitert)
test_data = [
    # freundlich
    {"sentence": "Du bist so toll und hilfst mir immer!",               "label": "freundlich"},
    {"sentence": "Hallo lieber Dorfbewohner, könnte man preislich bei dir noch was machen? Ich kann dir 2 Rohes Kaninchen für einen Smaragd geben. Das wäre wirklich freundlich und ich würde mich echt freuen",        "label": "freundlich"},
    {"sentence": "Ich freue mich auf unsere Zusammenarbeit.",          "label": "freundlich"},
    {"sentence": "Dein Engagement wird sehr geschätzt.",              "label": "freundlich"},
    {"sentence": "Herzlichen Glückwunsch zu deinem großartigen Erfolg!", "label": "freundlich"},

    # neutral
    {"sentence": "Kann man preislich bei dir noch was machen? Ich kann dir 2 Rohes Kaninchen für einen Smaragd geben.",           "label": "neutral"},
    {"sentence": "Bitte sende mir bis Freitag den Abschlussbericht.",  "label": "neutral"},
    {"sentence": "Die Datei findest du im Anhang der E-Mail.",         "label": "neutral"},
    {"sentence": "Unsere nächste Station ist der Hauptbahnhof.",       "label": "neutral"},
    {"sentence": "Das Wetter soll morgen regenfrei bleiben.",          "label": "neutral"},

    # unfreundlich
    {"sentence": "Was für ein Idiot bist du bitte?",                   "label": "unfreundlich"},
    {"sentence": "Willst du stress du Arschloch?",          "label": "unfreundlich"},
    {"sentence": "Hey du arschloch, was kostet eine dämliche Karte bei dir?",                  "label": "unfreundlich"},
    {"sentence": "Halte dein Maul halt mal, ja?",                "label": "unfreundlich"},
    {"sentence": "Komm raus ich hau dich!",   "label": "unfreundlich"},
    {"sentence": "Du Spast, wie ist das Wetter?",                "label": "unfreundlich"},
]

label_list = ["unfreundlich", "neutral", "freundlich"]
label2id   = {lab:i for i,lab in enumerate(label_list)}
id2label   = {i:lab for lab,i in label2id.items()}

# 2) Dataset erzeugen und Labels mappen
test_ds = Dataset.from_list(test_data)
test_ds = test_ds.map(lambda ex: {"labels": label2id[ex["label"]]}, remove_columns=["label"])

# 3) Tokenizer & Tokenisierung
MODEL_NAME = "dvm1983/TinyBERT_General_4L_312D_de"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
def tokenize_fn(ex):
    return tokenizer(ex["sentence"], padding="max_length", truncation=True, max_length=128)
test_ds = test_ds.map(tokenize_fn, batched=True, remove_columns=["sentence"])
test_ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

# 4) Data collator
data_collator = DataCollatorWithPadding(tokenizer)

# 5) compute_metrics: nur Accuracy
def compute_metrics(p):
    logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds  = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

# 6) Evaluierungs-Funktion
def evaluate_model(model_path: str, name: str):
    print(f"\n--- Evaluation: {name} ---")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Modell laden
    model = AutoModelForSequenceClassification.from_pretrained(
        model_path,
        num_labels=len(label_list),
        id2label=id2label,
        label2id=label2id
    ).to(device)
    model.eval()

    # Dataloader
    test_loader = DataLoader(test_ds, batch_size=16)

    all_preds = []
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=-1)
            preds = torch.argmax(probs, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Originaldaten für Anzeige
    sentences = [ex["sentence"] for ex in test_data]

    # Anzeige
    print("\nEinzelne Predictions:")
    for i, (sent, true_id, pred_id, prob_row) in enumerate(zip(sentences, all_labels, all_preds, all_probs)):
        print(f"{i+1:2d}. „{sent}“")
        print(f"     → True: {id2label[true_id]:11s} ({true_id}),"
              f"  Pred: {id2label[pred_id]:11s} ({pred_id}),"
              f"  Conf: {prob_row[pred_id]:.2f}")

    # Gesamtbewertung
    print("\nGesamtbewertung:")
    print(classification_report(all_labels, all_preds, target_names=label_list))

# 7) Vergleich
evaluate_model(MODEL_NAME, "Original_TinyBERT")
evaluate_model("../tinybert-german-finetuned", "Finetuned_TinyBERT")


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]


--- Evaluation: Original_TinyBERT ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dvm1983/TinyBERT_General_4L_312D_de and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi


Einzelne Predictions:
 1. „Du bist so toll und hilfst mir immer!“
     → True: freundlich  (2),  Pred: unfreundlich (0),  Conf: 0.37
 2. „Hallo lieber Dorfbewohner, könnte man preislich bei dir noch was machen? Ich kann dir 2 Rohes Kaninchen für einen Smaragd geben. Das wäre wirklich freundlich und ich würde mich echt freuen“
     → True: freundlich  (2),  Pred: unfreundlich (0),  Conf: 0.36
 3. „Ich freue mich auf unsere Zusammenarbeit.“
     → True: freundlich  (2),  Pred: unfreundlich (0),  Conf: 0.38
 4. „Dein Engagement wird sehr geschätzt.“
     → True: freundlich  (2),  Pred: unfreundlich (0),  Conf: 0.40
 5. „Herzlichen Glückwunsch zu deinem großartigen Erfolg!“
     → True: freundlich  (2),  Pred: unfreundlich (0),  Conf: 0.38
 6. „Kann man preislich bei dir noch was machen? Ich kann dir 2 Rohes Kaninchen für einen Smaragd geben.“
     → True: neutral     (1),  Pred: unfreundlich (0),  Conf: 0.36
 7. „Bitte sende mir bis Freitag den Abschlussbericht.“
     → True: neutral    

### Model zu Onnx konvertieren, damit es in Javascript eingebunden werden kann

In [26]:
%pip install "optimum[exporters]" onnxruntime

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [27]:
%%bash
mkdir -p tinybert-german-finetuned/onnx
optimum-cli export onnx \
  --model tinybert-german-finetuned \
  --task sequence-classification \
  tinybert-german-finetuned/onnx


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

